<a href="https://colab.research.google.com/github/temmyzeus/Tensorflow-Courses/blob/master/DeepLearning.AI%20TensorFlow%20Developer%20Professional%20Certificate/Natural%20Language%20Processing%20with%20Tensorflow/IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow_datasets.text import IMDBReviews